# Converting to TensorFlow Lite using pytorch_to_keras

This notebook shows how to convert mobilenet_v2 from a pytorch model into a quantized TensorFlow Lite model. First, use the python library `pytorch2keras` to convert the model into a Keras model, then follow the usual steps to export from Keras as a quantised int8 tflite model.

Ensure that you have installed Python 3.8 and have the installed ../requirements.txt

In [2]:
import sys
import os

# allow importing helper functions from local module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# The original libraries are unable to convert Relu6 operations
!{sys.executable} -m pip install https://github.com/banoffee-pie/onnx2keras/archive/refs/heads/master.zip
!{sys.executable} -m pip install https://github.com/banoffee-pie/pytorch2keras/archive/refs/heads/master.zip

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     \ 59 kB 1.2 MB/ssB/s
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     | 54 kB 805 kB/ssB/s


In [4]:
import torch
import io, os, shutil
import tensorflow as tf
import numpy as np
import tflite
from pytorch2keras import pytorch_to_keras
from torch.autograd import Variable

2023-06-23 14:37:19.582207: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's 

## Import PyTorch Model
For this example, we use mobilenet_v2.

In [5]:
pytorch_model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# Switch the model to eval mode
pytorch_model.eval()

Using cache found in /Users/salmankhan/.cache/torch/hub/pytorch_vision_v0.10.0


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

## Run Inference on PyTorch Model

First, lets run inference on the PyTorch model directly, just to see how it works.

In [6]:
# Download an image to test against
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

import requests
# Download Image Labels
resp = requests.get("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt")
# Read the categories
categories = [s.strip() for s in resp.text.splitlines()]

In [7]:
# We will test and train with these params
batch_size = 1
channels = 3
height = 224
width = 224

In [8]:
from PIL import Image
from torchvision import transforms

# Open testing image
input_image = Image.open(filename)

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(height),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Note Pytorch is BCHW
input_tensor = preprocess(input_image)

In [9]:
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

with torch.no_grad():
    output = pytorch_model(input_batch)

probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Show top categories per image
vals, idxs = torch.topk(probabilities, 5)
pytorch_results = [(categories[idx], prob) for (idx, prob) in zip(idxs.tolist(), vals.tolist())]
for cat, prob in  pytorch_results:
    print(cat, ':', prob)

Samoyed : 0.8303043246269226
Pomeranian : 0.06988773494958878
keeshond : 0.012964080087840557
collie : 0.010797776281833649
Great Pyrenees : 0.009886783547699451


## Convert to Keras

In [10]:
def pytorch_to_keras_model(pytorch_model, input_shape) -> tf.keras.Model:
    input_np = np.random.uniform(0, 1, tuple([ 1 ]) + input_shape)
    input_var = Variable(torch.FloatTensor(input_np))

    return pytorch_to_keras(
        pytorch_model,
        input_var,
        [input_shape],
        verbose=True,
        name_policy='renumerate',
        change_ordering=True # change channel_first to channel_last
    )

In [11]:
keras_model = pytorch_to_keras_model(pytorch_model, input_tensor.shape)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[torch.Size([3, 224, 224])]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight classifier.1.weight with shape (1000, 1280).
DEBUG:onnx2keras:Found weight classifier.1.bias with shape (1000,).
DEBUG:onnx2keras:Found weight onnx::Conv_538 with shape (32, 3, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_539 with shape (32,).
DEBUG:onnx2keras:Found weight onnx::Conv_541 with shape (32, 1, 3, 3).
DEBUG:onnx2keras:Found weight onnx::Conv_542 with shape (32,).
DEBUG:onnx2keras:Found weight onnx::Conv_544 with shape (16, 32, 1, 1).
DEBUG:onnx2keras:Found weight onnx::Conv

Exported graph: graph(%input_0 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %classifier.1.weight : Float(1000, 1280, strides=[1280, 1], requires_grad=1, device=cpu),
      %classifier.1.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_538 : Float(32, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_539 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_541 : Float(32, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_542 : Float(32, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_544 : Float(16, 32, 1, 1, strides=[32, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_545 : Float(16, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_547 : Float(96, 16, 1, 1, strides=[16, 1, 1, 1], requires_grad=0, device=cpu),
      %onnx::Conv_548 : Float(96, strides=[1], requires_grad=0, device=cpu),
      %onnx

DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Constant
DEBUG:onnx2keras:node_name: onnx::Clip_318
DEBUG:onnx2keras:node_params: {'value': array(6., dtype=float32), 'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Clip
DEBUG:onnx2keras:node_name: onnx::Conv_319
DEBUG:onnx2keras:node_params: {'change_ordering': True, 'name_policy': 'renumerate'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name input.4).
DEBUG:onnx2keras:Check input 1 (name onnx::Clip_317).
DEBUG:onnx2keras:Check input 2 (name onnx::Clip_318).
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras.clip:Using ReLU(6) instead of cl

### Check keras conversion

In [12]:
def softmax(xs):
    return np.exp(xs)/sum(np.exp(xs))

#transpose the input_batch into BHWC order for tensorflow
tf_input_data = np.transpose( input_batch.numpy(), [0, 2, 3, 1])

keras_output_data = keras_model(tf_input_data)

probs = keras_output_data[0]
data = zip(range(len(probs)), probs)
keras_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  keras_results:
    print(cat, ':', prob)

Samoyed : tf.Tensor(14.355221, shape=(), dtype=float32)
Pomeranian : tf.Tensor(11.880313, shape=(), dtype=float32)
keeshond : tf.Tensor(10.195609, shape=(), dtype=float32)
collie : tf.Tensor(10.012769, shape=(), dtype=float32)
Great Pyrenees : tf.Tensor(9.924629, shape=(), dtype=float32)


## Convert to tflite

We will still feed the data into the model in float32 format for convinence but the internals of the model will be int8. This will require representitive data but as we interface in float32 we can use the pytorch preprocessing. 

This conversion follows the method from [keras_to_xcore.ipynb](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb)

### Representative Dataset
To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds

ds = tfds.load('imagenette', split='train', as_supervised=True, shuffle_files=True).shuffle(1000).batch(1).prefetch(10).take(1000)

# Iterate over the sampled images and preprocess them
def representative_dataset():
    for image, _ in ds:
        pil_img = tf.keras.utils.array_to_img(image[0])
        pytorch_batch = preprocess(pil_img).unsqueeze(0)
        tf_batch = np.transpose(pytorch_batch.numpy(), [0, 2, 3, 1])
        yield [tf_batch]

INFO:absl:No config specified, defaulting to config: imagenette/full-size-v2
INFO:absl:Load dataset info from /Users/salmankhan/tensorflow_datasets/imagenette/full-size-v2/1.0.0
INFO:absl:Reusing dataset imagenette (/Users/salmankhan/tensorflow_datasets/imagenette/full-size-v2/1.0.0)
INFO:absl:Constructing tf.data.Dataset imagenette for split train, from /Users/salmankhan/tensorflow_datasets/imagenette/full-size-v2/1.0.0


### Conversion Process

In [14]:
# Now do the conversion to int8
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

tflite_int8_model = converter.convert()

# Save the model.
tflite_int8_model_path = 'mobilenet_v2.tflite'
with open(tflite_int8_model_path, 'wb') as f:
  f.write(tflite_int8_model)

INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpglftuy_k/assets


INFO:tensorflow:Assets written to: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpglftuy_k/assets
/usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-06-23 14:40:21.887270: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-23 14:40:21.887292: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-23 14:40:21.887904: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/fg/_pf9q2tj3cl9yfjb392zl7rm0000gn/T/tmpglftuy_k
2023-06-23 14:40:21.900155: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-23 14:40:21.900178: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) 

### Run inference

In [15]:
tfl_interpreter = tf.lite.Interpreter(model_path=tflite_int8_model_path)
tfl_interpreter.allocate_tensors()

tfl_input_details = tfl_interpreter.get_input_details()
tfl_output_details = tfl_interpreter.get_output_details()

# Convert PyTorch Input Tensor into Numpy Matrix and Reshape for TensorFlow
tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_input_data)
tfl_interpreter.invoke()

tfl_output_data = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

probs = softmax(tfl_output_data[0])
data = zip(range(len(probs)), probs)
tfl_int8_results = [(categories[idx], prob) for (idx, prob) in sorted(data, key=lambda x: x[1], reverse=True)[:5]]
for cat, prob in  tfl_int8_results:
    print(cat, ':', prob)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Samoyed : 0.8413713
Pomeranian : 0.06962144
West Highland white terrier : 0.022041732
keeshond : 0.010238749
white wolf : 0.006978273


### Analyse Model

In [16]:
import utils

utils.print_operator_counts(tflite_int8_model)

OPERATOR              COUNT
-------------------- ------
quantize                  1
pad                      18
conv_2d                  35
depthwise_conv_2d        17
add                      10
mean                      1
expand_dims               2
transpose                 1
reshape                   1
fully_connected           1
dequantize                1
-------------------- ------
TOTAL                    88
-------------------- ------


### Accuracy

In [17]:
import tensorflow_datasets as tfds
import tensorflow as tf
import requests
from typing import List

# load dataset
ds, info = tfds.load('imagenet_v2', split='test', with_info=True, as_supervised=True, shuffle_files=False)
ds = ds.shuffle(100, reshuffle_each_iteration=True)    

In [18]:
def accuracy_tflite(top_n:int = 1, samples=1000, verbose=False) -> float:
    if top_n < 1 or n > 1000:
        raise ValueError
    
    # take subset of dataset
    selection = ds.prefetch(10).take(samples)

    correct = 0
    incorrect = 0
    
    for image, label in selection: 
        trueCatIdx = tf.get_static_value(label)
        # convert to PIL.Image
        img = tf.keras.utils.array_to_img(image)
        
        # preprocess using PyTorch functions then convert back into Tf.Tensor
        pytorch_batch = preprocess(img).unsqueeze(0)
        tf_batch = np.transpose( pytorch_batch.numpy(), [0, 2, 3, 1])

        # use same tflite interpreter as before
        tfl_interpreter.set_tensor(tfl_input_details[0]['index'], tf_batch)
        tfl_interpreter.invoke()

        output = tfl_interpreter.get_tensor(tfl_output_details[0]['index'])

        # Sort into List[Tuple[index, confidence]] ordered by confidence (descending)
        data = sorted(
            zip(range(len(output[0])), output[0]),
            key=lambda x: x[1], reverse=True
        )

        top_n_results: List[int] = [idx for (idx, _) in data[:top_n]]

        if trueCatIdx in top_n_results:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
            if(verbose):
                print("--incorrect--")
                print(f"True Category: {categories[trueCatIdx]}({trueCatIdx})")
                print([f"Top-{top_n} categories: {categories[idx]}({idx})" for idx in top_n_results])
                display(img)
    
    accuracy = (correct / (correct+incorrect))
    print(f"Top-{top_n} accuracy (TFLite Model): {accuracy * 100}% ({correct}/{correct+incorrect})")
    return accuracy

In [19]:
def accuracy_torch(top_n: int = 1, samples=1000, verbose=False):
    if top_n < 1 or n > 1000:
        raise ValueError
    
    # take subset of dataset
    selection = ds.prefetch(10).take(samples)

    correct = 0
    incorrect = 0
    
    for image, label in selection: 
        trueCatIdx = tf.get_static_value(label)
        
        # convert to PIL.Image
        img = tf.keras.utils.array_to_img(image)
        input_batch = preprocess(img).unsqueeze(0)
        
        with torch.no_grad():
            output = pytorch_model(input_batch)

        # Show top categories per image
        vals, idxs = torch.topk(output[0], top_n)
        
        if trueCatIdx in idxs:
            correct = correct + 1
        else:
            incorrect = incorrect + 1
            if(verbose):
                print("--incorrect--")
                print(f"True Category: {categories[trueCatIdx]}({trueCatIdx})")
                print([f"Top-{top_n} categories: {categories[idx]}({idx})" for idx in top_n_results])
                display(img)
    
    accuracy = (correct / (correct+incorrect))
    print(f"Top-{top_n} accuracy (PyTorch Model): {accuracy * 100}% ({correct}/{correct+incorrect})")
    return accuracy

In [ ]:
samples = 500
for n in range(5):
    accuracy_torch(n+1, samples)
    accuracy_tflite(n+1, samples)

2023-06-23 14:42:00.436903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [16]
	 [[{{node Placeholder/_3}}]]
2023-06-23 14:42:00.437307: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [16]
	 [[{{node Placeholder/_0}}]]


Top-1 accuracy (PyTorch Model): 54.800000000000004% (274/500)


2023-06-23 14:42:12.252693: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-06-23 14:42:12.253210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [16]
	 [[{{node Placeholder/_1}}]]


Top-1 accuracy (TFLite Model): 53.800000000000004% (269/500)


2023-06-23 14:42:20.745678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [16]
	 [[{{node Placeholder/_4}}]]
2023-06-23 14:42:20.746144: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [16]
	 [[{{node Placeholder/_2}}]]
